<a href="https://colab.research.google.com/github/simbakm/training_yolov11_notebook/blob/main/examples/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **starting Point @ simbakm Training YOlo  version 11 bold text**

# Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://clickpy.clickhouse.com/dashboard/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
!uv pip install ultralytics
!pip install roboflow kagglehub
import ultralytics
import os
import shutil
from glob import glob
from tqdm import tqdm
import json
import random
from ultralytics import YOLO
ultralytics.checks()



Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.8/112.6 GB disk)


**STEP 2 — Download the Roboflow dataset (YOLOv11-ready)bold text**

**helper fuction to couint the number of files**

In [2]:
import os
from collections import defaultdict

def count_file_types(dataset_path):
    file_counts = defaultdict(int)

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            ext = os.path.splitext(file)[1].lower()  # get file extension
            file_counts[ext] += 1

    print("File type counts:\n")
    for ext, count in sorted(file_counts.items()):
        print(f"{ext if ext else 'No Extension'} : {count}")

    return dict(file_counts)


In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="YdrZFjwWblcZBkXrqqaR")
project = rf.workspace("stairs-ttqqp").project("trash-bin-asn0s")
dataset = project.version(1).download("yolov11")

roboflow_path = dataset.location
print("Roboflow dataset:", roboflow_path)

counts = count_file_types(roboflow_path)



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to trash-bin-1 in yolov11:: 100%|██████████| 902/902 [00:00<00:00, 8935.03it/s]

Roboflow dataset: /content/trash-bin-1
File type counts:

.jpg : 445
.txt : 447
.yaml : 1


## **STEP 3: Download the TACO dataset from KaggleHub**

In [4]:
import kagglehub
path = kagglehub.dataset_download("kneroma/tacotrashdataset")
print("Downloaded to:", path)
counts = count_file_types(path)


100%|██████████| 2.79G/2.79G [02:09<00:00, 23.1MB/s]

Extracting files...


Downloaded to: /root/.cache/kagglehub/datasets/kneroma/tacotrashdataset/versions/3
File type counts:

.bin : 1
.csv : 1
.jpg : 1500
.json : 1
.txt : 1


**STEP 4 — Convert TACO (COCO JSON) → YOLO format**

In [5]:


# TACO dataset path (from KaggleHub)
taco_path = path
data_dir = os.path.join(taco_path, "data")

# Load annotations
annotations_path = os.path.join(data_dir, "annotations.json")
annotations = json.load(open(annotations_path))

# Output
output_taco = "/content/taco_yolo"
#delete privous derectories
if os.path.exists(output_taco):
    shutil.rmtree(output_taco)
    print("Old taco_yolo directory removed.")
else:
    print("No previous taco_yolo folder found.")

os.makedirs(f"{output_taco}/images", exist_ok=True)
os.makedirs(f"{output_taco}/labels", exist_ok=True)

# Map image id → image info
image_map = {img["id"]: img for img in annotations["images"]}

# Find all images recursively
all_images = {}
image_files = glob(f"{data_dir}/**/*.jpg", recursive=True) + \
              glob(f"{data_dir}/**/*.JPG", recursive=True)

for f in image_files:
    all_images[os.path.relpath(f, data_dir)] = f


print("Total images found:", len(all_images))

# Group annotations by image ID
ann_by_image = {}
for ann in annotations["annotations"]:
    img_id = ann["image_id"]
    ann_by_image.setdefault(img_id, []).append(ann)

# Convert to YOLO
for img_id, img_info in tqdm(image_map.items()):
    filename = img_info["file_name"]
    batch_relative_path = filename  # contains batch_x/000000.jpg

    if batch_relative_path not in all_images:
        print("Missing image:", batch_relative_path)
        continue

    src = all_images[batch_relative_path]

    # create unique new filename based on img_id
    new_name = f"{img_id}.jpg"
    dst = f"{output_taco}/images/{new_name}"

    shutil.copy(src, dst)

    # image width/height
    W = img_info["width"]
    H = img_info["height"]

    # label path (same name as image)
    label_path = f"{output_taco}/labels/{img_id}.txt"
    with open(label_path, "w") as f:
        for ann in ann_by_image.get(img_id, []):
            x, y, w, h = ann["bbox"]

            # convert to YOLO
            xc = (x + w/2) / W
            yc = (y + h/2) / H
            w /= W
            h /= H
            # Create category id mapping
            categories = annotations["categories"]
            cat_id_map = {cat["id"]: i for i, cat in enumerate(categories)}
            class_id = cat_id_map[ann["category_id"]]

            f.write(f"{class_id} {xc} {yc} {w} {h}\n")

print("DONE — TACO converted to YOLO:", output_taco)


No previous taco_yolo folder found.
Total images found: 1500


100%|██████████| 1500/1500 [00:17<00:00, 86.91it/s] 

DONE — TACO converted to YOLO: /content/taco_yolo


# **5: Split the dataset into Train,val and test using ratio 80|10|10**

In [6]:
import os, random, shutil
from glob import glob

taco_img = "/content/taco_yolo/images"
taco_lbl = "/content/taco_yolo/labels"

images = sorted(glob(f"{taco_img}/*.jpg"))
random.shuffle(images)

n = len(images)
train_split = int(n * 0.8)
val_split = int(n * 0.9)

train_files = images[:train_split]
val_files   = images[train_split:val_split]
test_files  = images[val_split:]

def copy_split(files, folder):
    os.makedirs(f"/content/taco_split/images/{folder}", exist_ok=True)
    os.makedirs(f"/content/taco_split/labels/{folder}", exist_ok=True)
    for img in files:
        lbl = img.replace("images", "labels").replace(".jpg", ".txt")
        shutil.copy(img, f"/content/taco_split/images/{folder}/")
        shutil.copy(lbl, f"/content/taco_split/labels/{folder}/")

copy_split(train_files, "train")
copy_split(val_files, "val")
copy_split(test_files, "test")

print("TACO split complete!")


TACO split complete!


**Download COCO (80 Classes)**

In [7]:
coco_path = kagglehub.dataset_download("awsaf49/coco-2017-dataset")
print("COCO downloaded to:", coco_path)

Using Colab cache for faster access to the 'coco-2017-dataset' dataset.
COCO downloaded to: /kaggle/input/coco-2017-dataset


**Convert COCO to YOLO Format**

In [8]:
from ultralytics.data.converter import convert_coco

convert_coco(
    labels_dir=f"{coco_path}/annotations",
    save_dir="/content/coco_yolo",
    use_segments=False
)

print("COCO converted to YOLO format.")


COCO data converted successfully.
Results saved to /content/coco_yolo
COCO converted to YOLO format.


**Group TACO Classes into 10 & Shift IDs (COCO=0–79, TACO=80–89)**

In [9]:
taco_group_map = {
    0:0,1:0,2:0,3:0,      # Plastic
    4:1,5:1,              # Glass
    6:2,7:2,8:2,           # Metal
    9:3,10:3,              # Paper/Cardboard
    11:4,                   # Cigarette
    12:5,13:5,14:5,         # Food wrapper
    15:6,                   # Styrofoam
    16:7,17:7,              # Rubber/Textile
    18:8,19:8,              # Organic
    20:9,21:9,22:9          # Electronics/Other
}

TACO_OFFSET = 80

for split in ["train", "val", "test"]:
    lbl_files = glob(f"/content/taco_split/labels/{split}/*.txt")
    for file in lbl_files:
        new_lines = []
        with open(file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) > 0:
                    original_id = int(parts[0])
                    if original_id in taco_group_map:
                        grouped_id = taco_group_map[original_id]
                        new_id = grouped_id + TACO_OFFSET
                        parts[0] = str(new_id)
                        new_lines.append(" ".join(parts))
        with open(file, "w") as f:
            f.write("\n".join(new_lines))

print("TACO grouped into 10 classes and shifted.")


TACO grouped into 10 classes and shifted.


**STEP 6 — Create a combined dataset**

In [10]:
import glob

rf_labels = glob.glob(f"{roboflow_path}/**/labels/*.txt", recursive=True)

for file in rf_labels:
    lines = []
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 0:
                parts[0] = "90"  # move to class 90
                lines.append(" ".join(parts))

    with open(file, "w") as f:
        f.write("\n".join(lines))

print("Roboflow labels shifted to class 90")


Roboflow labels shifted to class 60


In [11]:
# remove old combined folder
shutil.rmtree("/content/combined", ignore_errors=True)

# create combined structure
for split in ["train", "val", "test"]:
    os.makedirs(f"/content/combined/images/{split}", exist_ok=True)
    os.makedirs(f"/content/combined/labels/{split}", exist_ok=True)

# copy TACO train/val/test
!cp -r /content/taco_split/images/train/* /content/combined/images/train/
!cp -r /content/taco_split/labels/train/* /content/combined/labels/train/

!cp -r /content/taco_split/images/val/* /content/combined/images/val/
!cp -r /content/taco_split/labels/val/* /content/combined/labels/val/

!cp -r /content/taco_split/images/test/* /content/combined/images/test/
!cp -r /content/taco_split/labels/test/* /content/combined/labels/test/

# Copy all COCO images to combined train/val
coco_images = glob.glob("/content/coco_yolo/images/*.jpg")
for img in coco_images:
    if random.random() < 0.8:
        shutil.copy(img, "/content/combined/images/train/")
        lbl = img.replace("/images/","/labels/").replace(".jpg",".txt")
        shutil.copy(lbl, "/content/combined/labels/train/")
    else:
        shutil.copy(img, "/content/combined/images/val/")
        lbl = img.replace("/images/","/labels/").replace(".jpg",".txt")
        shutil.copy(lbl, "/content/combined/labels/val/")

# copy roboflow
# train
!cp -r $roboflow_path/train/images/* /content/combined/images/train/
!cp -r $roboflow_path/train/labels/* /content/combined/labels/train/

# val
!cp -r $roboflow_path/valid/images/* /content/combined/images/val/
!cp -r $roboflow_path/valid/labels/* /content/combined/labels/val/

# test
!cp -r $roboflow_path/test/images/* /content/combined/images/test/
!cp -r $roboflow_path/test/labels/* /content/combined/labels/test/



**step7: Create YOLO data.yaml**

In [12]:


# Load pretrained YOLO
model = YOLO("yolo11n.pt")

# Convert COCO class dict → ordered list
coco_names = [model.names[i] for i in range(len(model.names))]

# TACO grouped classes (10 classes)
taco_grouped_classes = [
    "plastic","glass","metal","paper","cigarette",
    "food_wrapper","styrofoam","rubber_textile",
    "organic","electronics_other"
]

# Trash bin class
final_names = coco_names + taco_grouped_classes + ["trash_bin"]

# Write combined.yaml
with open("combined.yaml", "w") as f:
    f.write("train: /content/combined/images/train\n")
    f.write("val: /content/combined/images/val\n")
    f.write("test: /content/combined/images/test\n\n")
    f.write(f"nc: {len(final_names)}\n")
    f.write("names:\n")
    for name in final_names:
        f.write(f"  - {name}\n")

print("combined.yaml created with", len(final_names), "classes.")


combined.yaml created with 91 classes.


# **8: — Train YOLOv11**

In [15]:
from ultralytics import YOLO

# Path to your combined YAML
data_yaml = "combined.yaml"

# Use YOLO11n pretrained on COCO as base
model = YOLO("yolo11n.pt")

# Training parameters
model.train(
    data=data_yaml,        # 91-class dataset
    epochs=100,            # max epochs
    imgsz=640,             # image size
    batch=16,              # adjust to GPU memory
    device=0,              # GPU index
    patience=10,           # early stopping if val mAP not improving for 10 epochs
    save=True,             # saves best.pt automatically
    pretrained=True,       # start from COCO weights
    optimizer='auto',      # YOLO chooses best optimizer
    lr0=0.01,              # initial learning rate (overridden if optimizer=auto)
    augment=True,          # enables basic augmentations
    auto_augment='randaugment', # stronger augmentations
    mosaic=1.0,            # mosaic augmentation
    mixup=0.1,             # mixup augmentation
    flipud=0.5,            # vertical flip
    fliplr=0.5,            # horizontal flip
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,  # color jitter
    deterministic=True,    # reproducible results
    save_period=5,         # save weights every 5 epochs
    plots=True             # generate loss/metrics plots automatically
)


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=combined.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, po

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7db2a0287b90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.

## **STEP 6 — Download trained model**

In [14]:
from google.colab import files
files.download("/content/runs/detect/train/weights/best.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>